In [1]:
import h5py
import numpy as np
import os

In [2]:
def convert_to_h5():
     data_dir = './dataset'
     total_np_x = list()
     total_np_y = list()
     total_np_log_x = list()
     save_path = f'./dataset/full_data.h5'
     import matplotlib.pyplot as plt
     import time
     from PIL import Image
     for dir in os.listdir(data_dir):
          count = 0
          for file in os.walk(os.path.join(data_dir, dir)):
               if len(file[2]) > 0:
                    print(file[0], len(file[2]))
                    for img_file in file[2]:
                         img_path = os.path.join(file[0], img_file)
                         img_f = Image.open(img_path)
                         orig_img = np.asarray(img_f)
                         resized = Image.fromarray(orig_img).resize(size=(80, 80))
                         resized_array = np.asarray(resized)
                         resized_squeezed = resized_array.reshape(80*80*3, 1)
                         resized_log = Image.fromarray(resized_array).resize(size = (2, 2))
                         resized_log_squeezed = np.asarray(resized_log)
                         resized_log_squeezed = resized_log_squeezed.reshape(2*2*3, 1)
                         # print(resized_squeezed.shape)
                         total_np_x.append(resized_squeezed)
                         total_np_log_x.append(resized_log_squeezed)
                         if 'cat' in img_file:
                              total_np_y.append(0)
                         else:
                              total_np_y.append(1)
               
                         count += 1
               if count % 1000:
                    print(count)
               # break
          # break

     total_np_x = np.squeeze(np.asarray(total_np_x), axis = 2)
     total_np_y = np.squeeze(np.asarray(total_np_y))
     print(np.asarray(total_np_log_x).shape)
     total_np_log_x = np.squeeze(np.asarray(total_np_log_x), axis = 2)
     hf = h5py.File(save_path, 'a') 
     grp_x = hf.create_dataset('data_x', data = total_np_x)
     grp_y = hf.create_dataset('data_y', data = total_np_y)
     grp_x_log = hf.create_dataset('data_x_log', data = total_np_log_x)

     hf.close()
     
     # print('hdf5 file size: %d bytes'%os.path.getsize(save_path))

In [3]:
def load_data():
    total_dataset = h5py.File('./dataset/full_data3.h5', "r")

    # Test Set
    test_x_d = total_dataset["data_x"][1000:2000]
    test_y_d = total_dataset["data_y"][1000:2000]

    test_x_c = total_dataset["data_x"][:1000]
    test_y_c = total_dataset["data_y"][:1000]

    test_x = np.vstack((test_x_c, test_x_d))
    test_y = np.append(test_y_c, test_y_d)


    # Training Set
    train_x_d = total_dataset["data_x"][7000:10000]
    train_y_d = total_dataset["data_y"][7000:10000]

    train_x_c = total_dataset["data_x"][2000:5000]
    train_y_c = total_dataset["data_y"][2000:5000]

    train_y = np.append(train_y_c, train_y_d)
    train_x = np.vstack((train_x_c, train_x_d))


    train_x_d_log = total_dataset["data_x_log"][7000:10000]

    train_x_c_log = total_dataset["data_x_log"][2000:5000]

    train_x_log = np.vstack((train_x_c_log, train_x_d_log))
    # Validation Set

    valid_x_d = total_dataset["data_x"][6000:7000]
    valid_y_d = total_dataset["data_y"][6000:7000]

    valid_x_c = total_dataset["data_x"][5000:6000]
    valid_y_c = total_dataset["data_y"][5000:6000]

    valid_y = np.append(valid_y_c, valid_y_d)
    valid_x = np.vstack((valid_x_c, valid_x_d))

    # assert train_x.shape == (6000, 10800, 1)
    # assert test_x.shape == (2000, 10800, 1)
    # assert valid_x.shape == (2000, 10800, 1)
    
    return train_x, train_x_log, train_y, test_x, test_y, valid_x, valid_y




In [4]:
convert_to_h5()

./dataset\test_set\cats 1000
./dataset\test_set\dogs 1000
./dataset\training_set\cats 4000
./dataset\training_set\dogs 4000
(10000, 12, 1)
